# Imports

For loading and viewing your data, you will only need to import MNE. The additional line "%matplotlib qt" will ensure that your plots appear in pop-up windows rather than in the Jupyter notebook itself. This specification is important, as MNE's data viewer will only work properly when running in a separate window.

In [ ]:
import mne
%matplotlib qt

# Loading EEG Data

### EGI

In [ ]:
# Set the path to the current session's EEG recording
filepath = 'D://Documents/EEG_Workshop/data/Adult_08_T.mff'

# Load the EEG file and extract trigger times
eeg = mne.io.read_raw_egi(filepath, preload=True)
eeg.rename_channels({'E129': 'Cz'})
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
eeg = eeg.set_montage(montage)
events = mne.find_events(eeg)

### Biosemi

In [ ]:
# Set the path to the current session's EEG recording and digitization
filepath = 'D://Documents/EEG_Workshop/data/CTpitchTapEEG_sub01_block1.bdf'
montage_file = 'data/CTpitchTapEEG_sub01.sfp'

# Load the EEG file and extract trigger times
eeg = mne.io.read_raw_bdf(filepath, preload=True)
events = mne.find_events(eeg, initial_event=True)

# Load custom digitization file and apply to EEG object
# (Note that all non-data channels are dropped in order for montage to work)
montage = np.loadtxt(montage_file, dtype=str)
montage[:, 0] = [ch.replace('-', '') for ch in montage[:, 0]]
ch_names = montage[:, 0]
pos = montage[:, 1:].astype(float)
ch_pos = OrderedDict(zip(ch_names, pos))
nasion, rpa, lpa = [ch_pos.pop(ch, None) for ch in ('NZ', 'RPA', 'LPA')]
montage = mne.channels.make_dig_montage(ch_pos=ch_pos, coord_frame='unknown', nasion=nasion, rpa=rpa, lpa=lpa)
channels = ['A%s' % i for i in range(1, 33)] + ['B%s' % i for i in range(1, 33)] + \
            ['C%s' % i for i in range(1, 33)] + ['D%s' % i for i in range(1, 33)]
eeg = eeg.pick_channels(channels)
eeg = eeg.set_montage(montage)

# Viewing Sensor Locations
The following code can be used to plot either a 2D or 3D map of your EEG electrode locations. The most useful settings are explained below. Advanced users can view the full list of settings here: https://mne.tools/stable/generated/mne.io.Raw.html

### Settings:
- **kind**: Specifies whether the electrode locations should be plotted as a 2D topomap or a 3D head map. For 2D, set to 'topomap'. For 3D, set to '3d'.
- **show_names**: Set to True if you want to label each electrode with its name. Set to False to leave electrodes unlabeled. 
- **to_sphere**: This setting only matters if you are plotting as a topomap. Set to True to project channel locations to a sphere (this is the standard method of displaying a topomap). Set to False to display channels as if looking down on the cap from above.

In [ ]:
# Plot channel locations
eeg.plot_sensors(kind='3d', show_names=True, to_sphere=True)

# Plotting Continuous Data
The code below can be used to visualize the raw EEG traces from your recording. The most useful settings are explained below. Advanced users can view the full list of settings here: https://mne.tools/stable/generated/mne.io.Raw.html

Beyond acting as a data visualizer, the viewing panel also allows you to manually mark channels as bad by clicking anywhere along their data traces. Upon closing the viewer, any channels you marked as bad will be recorded in eeg.info['bads'].

### Settings:

The first two parameters specify how much data will appear in the window at one time.

- **n_channels**: Specifies how many channels will display in the window at once. (Can be ajdusted in the EEG viewer using Page Up/Page Down)
- **duration**: Specifies how many seconds of data will display in the window at once. (Can be ajdusted in the EEG viewer using Home/End)

The next two parameters give the option to plot only a subset of your data.

- **start**: Specifies how many seconds into the recording to start plotting, i.e. the start time of the plot. 
- **order**: Leave as None to plot all channels. Alternatively, specify a list of channel indices to only plot those channels in the order you provide.

The final two parameters affect the formatting of the plot.

- **butterfly**: Set to False to plot each channel on its own line. Set to True to plot all channels overlayed as a butterfly plot.
- **scalings**: Specifies the scaling factor for the EEG traces. Leave as None to use default scaling. Set to 'auto' for auto-scaling based on the voltage of the signal. Otherwise, set to a dictionary specifying the scaling for each channel type, e.g. MNE recommends dict(eeg=20e-6, eog=150e-6). (Can be ajdusted in the EEG viewer using +/-)

The plot() function also allows you to draw event markers over the EEG traces if you provide a matrix of event times. This matrix should have one row per event and three columns. The first column should list the index of the EEG sample corresponding to the start of each event, the second column can be ignored, and the third column should contain event ID numbers (if you have multiple types of events). To create an event matrix automatically from the EEG recording's trigger channel, you can run use the mne.find_events() function (shown in the code below).

In [ ]:
# Plot continuous data
eeg.plot(n_channels=32, 
         duration=10., 
         start=0.,
         order=None,
         butterfly=False,
         scalings=None,
         events=mne.find_events(eeg))

# Plotting Epoched Data

The method of plotting epoched data is very similar to plotting continuous data, just with slightly different parameters. The two code snippets below show 1) how to epoch your data using the trigger pulses in the EEG recording if you have not already done so, and 2) the function you can call to plot your epoched data.

Beyond simply acting as a data visualizer, the viewing panel also allows you to manually mark channels as bad by clicking their names on the Y axis and to mark epochs as bad by clicking one of the data traces within an epoch. Upon closing the viewer, any channels you marked as bad will be recorded in eeg.info['bads'] and any epochs you marked as bad will be dropped from your data.

Advanced users may want to reference the full list of plot settings here: https://mne.tools/stable/generated/mne.Epochs.html

### Settings:

First, there are two important settings when initially epoching your data.

- **tmin**: How long (in seconds) each epoch should extend before the onset of the event.
- **tmax**: How long (in seconds) each epoch should extend after the onset of the event.

Next, there are several settings to use when plotting epochs, which are similar to those used when plotting continuous data. There are two parameters which set how much data will appear in the EEG viewer at once.

- **n_channels**: Specifies how many channels will display in the window at once. (Can be ajdusted in the EEG viewer using Page Up/Page Down)
- **n_epochs**: Specifies how many epochs will display in the window at once. (Can be adjusted in the EEG viewer using Home/End)

There is one parameter that can be used if you wish to plot only a subset of channels.

- **picks**: Leave as None to plot all channels. Alternatively, specify a list of channel names/indices to only plot those channels in the order you provide.

Finally, there are two parameters that control the formatting of the plot.

- **butterfly**: Set to False to plot each channel on its own line. Set to True to plot all channels overlayed as a butterfly plot.
- **scalings**: Specifies the scaling factor for the EEG traces. Leave as None to use default scaling. Set to 'auto' for auto-scaling based on the voltage of the signal. Otherwise, set to a dictionary specifying the scaling for each channel type, e.g. MNE recommends dict(eeg=20e-6, eog=150e-6). (Can be adjusted in the EEG viewer using +/-)

In [ ]:
# Epoch data around the trigger pulses in the EEG recording
events = mne.find_events(eeg)
eeg = mne.Epochs(eeg, events, tmin=-.5, tmax=1., preload=True)

In [ ]:
# Plot epochs
eeg.plot(n_channels=20, 
         n_epochs=10, 
         picks=None, 
         butterfly=False, 
         scalings='auto')